In [1]:
import sys
print(sys.version)


3.6.9 (default, Oct  8 2020, 12:12:24) 
[GCC 8.4.0]


In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import datetime




In [3]:
from torch.utils.tensorboard import SummaryWriter

In [4]:
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader

In [5]:
train_transform = transforms.Compose([
                                      transforms.ToTensor()
])
valid_transform = train_transform

In [6]:
train_set = MNIST('./data/mnist', train=True, download=True, transform=train_transform)
valid_set = MNIST('./data/mnist', train=False, download=True, transform=valid_transform)




Extracting ./data/mnist/MNIST/raw/train-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


Extracting ./data/mnist/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw



Extracting ./data/mnist/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/mnist/MNIST/raw


Extracting ./data/mnist/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/mnist/MNIST/raw
Processing...


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [7]:
class myMLP(nn.Module):

  def __init__(self, N_input=1, N_output=10):
    super(myMLP, self).__init__()
    self.fc1 = nn.Linear(N_input, 128)
    self.fc2 = nn.Linear(128, 64)
    self.fc3 = nn.Linear(64, N_output)

  def forward(self, X):
    X = self.fc1(X)
    X = F.relu(X)
    X = self.fc2(X)
    X = F.relu(X)
    X = self.fc3(X)
    X = F.log_softmax(X, dim=1)
    return X

In [8]:
class LeNet(nn.Module):
  def __init__(self):
    super(LeNet, self).__init__()

    self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 6, 
                           kernel_size = 5, stride = 1, padding = 2)
    self.conv2 = nn.Conv2d(in_channels = 6, out_channels = 16, 
                           kernel_size = 5, stride = 1, padding = 0)
    self.conv3 = nn.Conv2d(in_channels = 16, out_channels = 120, 
                           kernel_size = 5, stride = 1, padding = 0)
    self.linear1 = nn.Linear(120, 84)
    self.linear2 = nn.Linear(84, 10)
    self.tanh = nn.Tanh()
    self.avgpool = nn.AvgPool2d(kernel_size = 2, stride = 2)

  def forward(self, x):
    x = self.conv1(x)
    x = self.tanh(x)
    x = self.avgpool(x)
    x = self.conv2(x)
    x = self.tanh(x)
    x = self.avgpool(x)
    x = self.conv3(x)
    x = self.tanh(x)
    
    x = x.reshape(x.shape[0], -1)
    x = self.linear1(x)
    x = self.tanh(x)
    x = self.linear2(x)
    return x
    

In [9]:
device = 'cuda'

In [10]:
N_input = 28 * 28
N_output = 10
# model = myMLP(N_input = N_input, N_output=10)
model = LeNet()
model.to(device)




LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
  (tanh): Tanh()
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)

In [11]:

print(model)


LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
  (tanh): Tanh()
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)


In [26]:
from torchsummary import summary
# summary(model, (1,784))
summary(model, (1,28,28))



----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              Tanh-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              Tanh-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
            Conv2d-7            [-1, 120, 1, 1]          48,120
              Tanh-8            [-1, 120, 1, 1]               0
            Linear-9                   [-1, 84]          10,164
             Tanh-10                   [-1, 84]               0
           Linear-11                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/ba

In [27]:
train_loader = torch.utils.data.DataLoader(train_set, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(valid_set, batch_size=16, shuffle=False)

In [14]:
def train(n_epochs, optimizer, model, loss_fn, train_loader, save_file = 'none' ):
  for epoch in range(1, n_epochs+1):
    loss_train = 0.0
    for imgs, labels in train_loader:

      imgs = imgs.to(device=device)
      labels = labels.to(device=device)

      # imgs = imgs.view(imgs.shape[0], -1)
      
      outputs = model(imgs)
      loss = loss_fn(outputs, labels)
      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
      loss_train += loss.item()
    
    scheduler.step(loss_train)

    # writer.add_scalar('training loss', loss_train/len(train_loader), epoch)
    
    print('{} Epoch {}, Training loss {}'.format(
        datetime.datetime.now(), epoch, loss_train/len(train_loader)))
    
    if save_file != 'none':
      torch.save(model.state_dict(), save_file)
    

In [28]:
def init_weights(m):
  if type(m) == nn.Linear:
    torch.nn.init.xavier_uniform_(m.weight)
    m.bias.data.fill_(0.01)

In [29]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, nesterov=True)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer,'min')
loss_fn = nn.CrossEntropyLoss()





In [17]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [18]:
# writer = SummaryWriter('./drive/MyDrive/tensorboard/runs/train')

In [19]:
# !kill 830

In [30]:
model.apply(init_weights)

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (conv3): Conv2d(16, 120, kernel_size=(5, 5), stride=(1, 1))
  (linear1): Linear(in_features=120, out_features=84, bias=True)
  (linear2): Linear(in_features=84, out_features=10, bias=True)
  (tanh): Tanh()
  (avgpool): AvgPool2d(kernel_size=2, stride=2, padding=0)
)

In [31]:
save_file = 'drive/MyDrive/checkpoint/LeNet-mnist-rev3'



In [32]:
train(n_epochs=10, optimizer=optimizer, model=model,loss_fn=loss_fn, train_loader=train_loader, save_file=save_file) 

2021-01-19 19:23:51.380904 Epoch 1, Training loss 0.038154138020255295
2021-01-19 19:24:07.702753 Epoch 2, Training loss 0.009995317349908873
2021-01-19 19:24:24.307890 Epoch 3, Training loss 0.005622892187279164


KeyboardInterrupt: ignored

In [23]:
def validate(model, train_loader, val_loader):
  for name, loader in [('train', train_loader),('val', val_loader)]:
    correct = 0
    total = 0
    with torch.no_grad():
      for imgs, labels in loader:
        imgs = imgs.to(device=device)
        labels = labels.to(device=device)
        # imgs = imgs.view(imgs.shape[0], -1)
        y = model(imgs)
        _, predicted = torch.max(y, dim=1)
        total += labels.shape[0]
        correct += int((predicted == labels).sum())

    print("Accuracy {}: {:.2f}".format(name, correct/total))
    print("{}: correct, total = {},{}".format(name, correct, total))



In [33]:
model.load_state_dict(torch.load(save_file))

<All keys matched successfully>

In [34]:

validate(model, train_loader, val_loader)

Accuracy train: 1.00
train: correct, total = 59972,60000
Accuracy val: 0.99
val: correct, total = 9911,10000


In [ ]:
# %load_ext tensorboard

In [ ]:
# %tensorboard --logdir ./drive/MyDrive/tensorboard/runs/train --host localhost --port 8088
